In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Acre - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [22]:
data = pd.read_csv('2003_mo_model_input_AC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado,Acre - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,1.746110,982.311732,...,17.406831,79.784179,1.371080,8.722906,0.691320,7.285707e+06,413409.255032,9.318417,6.587220e+06,3.260
1,2003-2,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,1.749146,982.322157,...,17.413702,79.765585,1.371489,8.718028,0.691617,7.294881e+06,413607.843560,9.319990,6.590338e+06,2.191
2,2003-3,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,1.750917,982.328239,...,17.417710,79.754739,1.371727,8.713149,0.691914,7.304055e+06,413806.432089,9.321563,6.593455e+06,2.154
3,2003-4,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,1.751107,982.328890,...,17.418139,79.753577,1.371752,8.708271,0.692212,7.313229e+06,414005.020618,9.323136,6.596572e+06,2.643
4,2003-5,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,1.758462,982.313560,...,17.452241,79.787637,1.384557,8.703393,0.692509,7.322403e+06,414203.609146,9.324709,6.599689e+06,2.546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.743830,989.336140,...,21.506209,61.729643,0.302367,NaN,NaN,NaN,NaN,NaN,NaN,21.748
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.533037,989.969068,...,21.474991,66.737103,0.399365,NaN,NaN,NaN,NaN,NaN,NaN,20.032
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.959645,990.394286,...,21.483613,70.109524,0.447539,NaN,NaN,NaN,NaN,NaN,NaN,18.573
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.882894,990.480242,...,21.364850,70.916992,0.454788,NaN,NaN,NaN,NaN,NaN,NaN,15.493


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
# mean = np.mean(input_data, axis=0)
# stddev =  np.std(input_data, axis=0)
# input_data = ((input_data - mean) /stddev)
scaler=MinMaxScaler()
scaled_data = scaler.fit_transform(input_data)
input_data = pd.DataFrame(scaled_data, columns=input_data.columns)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,1.000000,1.000000,0.813935,1.000000,0.000000,0.000000,1.000000,0.324035,0.247909,0.410954,0.000000,0.718774,0.835693,0.365169,0.056128,0.000000,0.000000,0.693537,0.455941,0.000000
1,0.921240,0.965719,0.698054,0.936603,0.007992,0.011007,0.968938,0.324785,0.248780,0.411383,0.001522,0.719058,0.835435,0.365319,0.054896,0.010205,0.006814,0.699191,0.464004,0.011752
2,0.873107,0.927412,1.000000,0.906543,0.015984,0.022015,0.948438,0.325223,0.249289,0.411633,0.002409,0.719223,0.835284,0.365407,0.053663,0.020410,0.013628,0.704845,0.472066,0.023504
3,0.851659,0.885588,0.629650,0.858675,0.023976,0.033022,0.928794,0.325270,0.249343,0.411660,0.002504,0.719241,0.835268,0.365416,0.052430,0.030616,0.020442,0.710499,0.480129,0.035256
4,0.823277,0.838034,0.795263,0.824839,0.031968,0.044030,0.970421,0.327087,0.248062,0.413268,0.006025,0.720649,0.835741,0.370125,0.051197,0.040821,0.027256,0.716153,0.488192,0.047008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.000000,0.030808,0.075260,0.381401,0.987600,0.828698,0.005731,0.715981,0.514335,0.835279,0.517627,0.877641,0.789886,0.501062,0.906664,0.977176,0.966200,0.097050,0.000701,0.793320
188,0.029504,0.027720,0.052572,0.466609,0.990700,0.823090,0.037761,0.598363,0.488155,0.881600,0.547254,0.894411,0.803293,0.709962,0.904491,0.973916,0.962712,0.102440,0.000526,0.789444
189,0.043125,0.023074,0.013826,0.530936,0.993800,0.817482,0.087044,0.546164,0.630912,0.844678,0.638567,0.905865,0.782672,0.695492,0.902319,0.970655,0.959224,0.107830,0.000351,0.785568
190,0.059756,0.017547,0.056080,0.611274,0.996900,0.811875,0.118264,0.486997,0.804373,0.879961,0.750573,0.905588,0.738310,0.681147,0.900147,0.967395,0.955735,0.113220,0.000175,0.781692


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      4.105
1      3.898
2      4.209
3      2.994
4      3.720
       ...  
235      NaN
236      NaN
237      NaN
238      NaN
239      NaN
Name: Acre - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
0,1.000000,1.000000,0.813935,1.000000,0.000000,0.000000,1.000000,0.324035,0.247909,0.410954,0.000000,0.718774,0.835693,0.365169,0.056128,0.000000,0.000000,0.693537,0.455941,0.000000
1,0.921240,0.965719,0.698054,0.936603,0.007992,0.011007,0.968938,0.324785,0.248780,0.411383,0.001522,0.719058,0.835435,0.365319,0.054896,0.010205,0.006814,0.699191,0.464004,0.011752
2,0.873107,0.927412,1.000000,0.906543,0.015984,0.022015,0.948438,0.325223,0.249289,0.411633,0.002409,0.719223,0.835284,0.365407,0.053663,0.020410,0.013628,0.704845,0.472066,0.023504
3,0.851659,0.885588,0.629650,0.858675,0.023976,0.033022,0.928794,0.325270,0.249343,0.411660,0.002504,0.719241,0.835268,0.365416,0.052430,0.030616,0.020442,0.710499,0.480129,0.035256
4,0.823277,0.838034,0.795263,0.824839,0.031968,0.044030,0.970421,0.327087,0.248062,0.413268,0.006025,0.720649,0.835741,0.370125,0.051197,0.040821,0.027256,0.716153,0.488192,0.047008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.379453,0.024030,0.365935,0.039971,0.890665,0.956993,0.182507,0.566576,0.567224,0.418651,0.454812,0.941438,0.919092,0.329691,0.995530,0.984319,0.992377,0.000007,0.085842,0.839039
158,0.332832,0.021531,0.401765,0.027548,0.893949,0.953662,0.168812,0.490277,0.624810,0.325951,0.379819,0.939108,0.944652,0.301838,0.991059,0.983545,0.993070,0.000014,0.080835,0.838516
159,0.313954,0.019385,0.379504,0.037038,0.897232,0.950331,0.152959,0.540787,0.680111,0.425407,0.309983,0.932751,0.961225,0.316094,0.986589,0.982772,0.993763,0.000021,0.075827,0.837993
160,0.294856,0.017671,0.440821,0.070841,0.900516,0.947000,0.137613,0.569819,0.736047,0.483364,0.269311,0.922555,0.959159,0.389642,0.982119,0.981998,0.994456,0.000028,0.070819,0.837470


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0       4.105
1       3.898
2       4.209
3       2.994
4       3.720
        ...  
157     9.081
158     9.545
159    10.130
160    14.451
161    13.218
Name: Acre - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 20)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Acre - Desemprego,Acre - IDH,Acre - PIB - Estadual,Acre - PIB - Construção Civil,Acre - PIB - Per Capita,Acre - PIB - Preços de Mercado
126,0.612440,0.052718,0.131587,0.201912,0.780557,0.994213,0.372054,0.513319,0.621789,0.639940,0.352790,0.808938,0.770832,0.464969,0.731313,0.712498,0.910689,0.374302,0.655667,0.994431
127,0.626227,0.056653,0.217266,0.209545,0.784414,0.995177,0.375438,0.582180,0.559112,0.674720,0.380305,0.819034,0.774301,0.474367,0.742775,0.713890,0.916766,0.367222,0.643610,0.995359
128,0.645442,0.060771,0.241706,0.238038,0.788271,0.996142,0.372254,0.566416,0.506188,0.660802,0.411712,0.852345,0.806922,0.480870,0.754236,0.715281,0.922843,0.360142,0.631553,0.996287
129,0.673151,0.065105,0.080099,0.249843,0.792128,0.997106,0.369458,0.563162,0.472667,0.565094,0.396481,0.875586,0.844584,0.479866,0.765698,0.716672,0.928919,0.353061,0.619495,0.997216
130,0.699949,0.069099,0.033180,0.266770,0.795984,0.998071,0.371547,0.570044,0.446018,0.506187,0.391500,0.889063,0.865439,0.505020,0.777159,0.718063,0.934996,0.345981,0.607438,0.998144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,0.000000,0.030808,0.075260,0.381401,0.987600,0.828698,0.005731,0.715981,0.514335,0.835279,0.517627,0.877641,0.789886,0.501062,0.906664,0.977176,0.966200,0.097050,0.000701,0.793320
188,0.029504,0.027720,0.052572,0.466609,0.990700,0.823090,0.037761,0.598363,0.488155,0.881600,0.547254,0.894411,0.803293,0.709962,0.904491,0.973916,0.962712,0.102440,0.000526,0.789444
189,0.043125,0.023074,0.013826,0.530936,0.993800,0.817482,0.087044,0.546164,0.630912,0.844678,0.638567,0.905865,0.782672,0.695492,0.902319,0.970655,0.959224,0.107830,0.000351,0.785568
190,0.059756,0.017547,0.056080,0.611274,0.996900,0.811875,0.118264,0.486997,0.804373,0.879961,0.750573,0.905588,0.738310,0.681147,0.900147,0.967395,0.955735,0.113220,0.000175,0.781692


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 20)

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
#     train, train_val = validation_splitter(train_input, 6)
#     target,target_val = validation_splitter(train_target, 6)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
#     optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(72, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(36, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(36, activation='tanh'),
        tf.keras.layers.Dense(18, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss='mean_squared_error')    
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
#                         validation_data=(train_val,
#                                          target_val),
                        validation_split=0.1,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3472618047, 1505302888, 8515080, 1940719957, 2927536597, 1624777467, 32034557, 3531691712, 1380127375, 3944213566]


Step: 0 ___________________________________________
val_loss: 5.486915111541748
winner_seed: 3472618047


Step: 1 ___________________________________________
val_loss: 5.435122966766357
winner_seed: 1505302888


Step: 2 ___________________________________________
val_loss: 2.929335355758667
winner_seed: 8515080


Step: 3 ___________________________________________
val_loss: 5.5113444328308105


Step: 4 ___________________________________________
val_loss: 5.432477951049805


Step: 5 ___________________________________________
val_loss: 5.5370707511901855


Step: 6 ___________________________________________
val_loss: 4.624905109405518


Step: 7 ___________________________________________


2023-10-19 20:01:36.370800: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 4.800616264343262


Step: 8 ___________________________________________
val_loss: 5.255267143249512


Step: 9 ___________________________________________


2023-10-19 20:03:00.039646: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-10-19 20:03:00.079099: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 2.859158992767334
winner_seed: 3944213566


final_seed: 3944213566


In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 116ms/step - loss: 197.2570 - val_loss: 126.3587
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 170.8447 - val_loss: 102.0173
Epoch 3/10000
4/4 [==============================] - 0s 7ms/step - loss: 147.0279 - val_loss: 83.8481
Epoch 4/10000
4/4 [==============================] - 0s 7ms/step - loss: 129.2149 - val_loss: 71.0870
Epoch 5/10000
4/4 [==============================] - 0s 7ms/step - loss: 115.5484 - val_loss: 61.5640
Epoch 6/10000
4/4 [==============================] - 0s 7ms/step - loss: 104.5390 - val_loss: 54.6151
Epoch 7/10000
4/4 [==============================] - 0s 7ms/step - loss: 96.3619 - val_loss: 49.5841
Epoch 8/10000
4/4 [==============================] - 0s 7ms/step - loss: 88.8782 - val_loss: 45.8582
Epoch 9/10000
4/4 [==============================] - 0s 7ms/step - loss: 84.6283 - val_loss: 42.9593
Epoch 10/10000
4/4 [==============================] - 0s 7ms/step - loss: 80.2414

4/4 [==============================] - 0s 8ms/step - loss: 23.6711 - val_loss: 4.3531
Epoch 82/10000
4/4 [==============================] - 0s 7ms/step - loss: 23.4138 - val_loss: 4.2482
Epoch 83/10000
4/4 [==============================] - 0s 7ms/step - loss: 23.1599 - val_loss: 4.1504
Epoch 84/10000
4/4 [==============================] - 0s 7ms/step - loss: 22.9365 - val_loss: 4.0574
Epoch 85/10000
4/4 [==============================] - 0s 7ms/step - loss: 22.6936 - val_loss: 3.9667
Epoch 86/10000
4/4 [==============================] - 0s 7ms/step - loss: 22.4727 - val_loss: 3.8771
Epoch 87/10000
4/4 [==============================] - 0s 8ms/step - loss: 22.2475 - val_loss: 3.7937
Epoch 88/10000
4/4 [==============================] - 0s 7ms/step - loss: 22.0073 - val_loss: 3.7144
Epoch 89/10000
4/4 [==============================] - 0s 7ms/step - loss: 21.7865 - val_loss: 3.6393
Epoch 90/10000
4/4 [==============================] - 0s 7ms/step - loss: 21.5777 - val_loss: 3.5685
Epoch

4/4 [==============================] - 0s 7ms/step - loss: 15.5549 - val_loss: 3.9224
Epoch 162/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.5357 - val_loss: 3.9526
Epoch 163/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.5114 - val_loss: 3.9773
Epoch 164/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.4993 - val_loss: 4.0066
Epoch 165/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.4860 - val_loss: 4.0390
Epoch 166/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.4641 - val_loss: 4.0662
Epoch 167/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.4458 - val_loss: 4.0939
Epoch 168/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.4345 - val_loss: 4.1263
Epoch 169/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.4220 - val_loss: 4.1541
Epoch 170/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.4134 - val_loss: 4.1

4/4 [==============================] - 0s 7ms/step - loss: 15.1499 - val_loss: 5.3114
Epoch 242/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1434 - val_loss: 5.3111
Epoch 243/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.1510 - val_loss: 5.3172
Epoch 244/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1432 - val_loss: 5.3225
Epoch 245/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1486 - val_loss: 5.3240
Epoch 246/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1500 - val_loss: 5.3203
Epoch 247/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1506 - val_loss: 5.3211
Epoch 248/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1465 - val_loss: 5.3272
Epoch 249/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1537 - val_loss: 5.3396
Epoch 250/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1347 - val_loss: 5.3

4/4 [==============================] - 0s 8ms/step - loss: 15.1385 - val_loss: 5.6560
Epoch 322/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1383 - val_loss: 5.6546
Epoch 323/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1367 - val_loss: 5.6433
Epoch 324/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.1487 - val_loss: 5.6417
Epoch 325/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.1382 - val_loss: 5.6375
Epoch 326/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1430 - val_loss: 5.6300
Epoch 327/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.1513 - val_loss: 5.6321
Epoch 328/10000
4/4 [==============================] - 0s 8ms/step - loss: 15.1412 - val_loss: 5.6439
Epoch 329/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1382 - val_loss: 5.6468
Epoch 330/10000
4/4 [==============================] - 0s 7ms/step - loss: 15.1424 - val_loss: 5.6

...

4/4 [==============================] - 0s 7ms/step - loss: 7.9995 - val_loss: 16.4562
Epoch 1121/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.7392 - val_loss: 16.5354
Epoch 1122/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.3915 - val_loss: 16.5184
Epoch 1123/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.8221 - val_loss: 16.5495
Epoch 1124/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.0149 - val_loss: 16.5686
Epoch 1125/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.7371 - val_loss: 16.5540
Epoch 1126/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.6051 - val_loss: 16.5054
Epoch 1127/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.7148 - val_loss: 16.4894
Epoch 1128/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.5626 - val_loss: 16.5131
Epoch 1129/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.5982 - val_l

4/4 [==============================] - 0s 7ms/step - loss: 7.4854 - val_loss: 17.2114
Epoch 1200/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.6520 - val_loss: 17.2595
Epoch 1201/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.4225 - val_loss: 17.3101
Epoch 1202/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.0141 - val_loss: 17.3716
Epoch 1203/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.3305 - val_loss: 17.4065
Epoch 1204/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.5798 - val_loss: 17.4091
Epoch 1205/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.9751 - val_loss: 17.3688
Epoch 1206/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.6987 - val_loss: 17.4265
Epoch 1207/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.9188 - val_loss: 17.4376
Epoch 1208/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.3818 - val_l

4/4 [==============================] - 0s 8ms/step - loss: 7.9070 - val_loss: 16.6722
Epoch 1279/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.4722 - val_loss: 16.6446
Epoch 1280/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.6962 - val_loss: 16.7364
Epoch 1281/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.8538 - val_loss: 16.8671
Epoch 1282/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.4755 - val_loss: 17.0940
Epoch 1283/10000
4/4 [==============================] - 0s 8ms/step - loss: 8.0285 - val_loss: 17.2397
Epoch 1284/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.7850 - val_loss: 17.2919
Epoch 1285/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.6237 - val_loss: 17.2556
Epoch 1286/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.9361 - val_loss: 17.2940
Epoch 1287/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.6682 - val_l

4/4 [==============================] - 0s 7ms/step - loss: 7.4380 - val_loss: 17.9437
Epoch 1358/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.3897 - val_loss: 17.9741
Epoch 1359/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.4402 - val_loss: 17.8713
Epoch 1360/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.8925 - val_loss: 17.7711
Epoch 1361/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.7553 - val_loss: 17.7002
Epoch 1362/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.0898 - val_loss: 17.5971
Epoch 1363/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.8217 - val_loss: 17.4901
Epoch 1364/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.7752 - val_loss: 17.3901
Epoch 1365/10000
4/4 [==============================] - 0s 8ms/step - loss: 7.4196 - val_loss: 17.4255
Epoch 1366/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.6577 - val_l

4/4 [==============================] - 0s 7ms/step - loss: 7.5775 - val_loss: 18.0612
Epoch 1437/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.5174 - val_loss: 17.9451
Epoch 1438/10000
4/4 [==============================] - 0s 7ms/step - loss: 8.2237 - val_loss: 18.1217
Epoch 1439/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.4287 - val_loss: 18.2652
Epoch 1440/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.5973 - val_loss: 18.3354
Epoch 1441/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.4045 - val_loss: 18.2467
Epoch 1442/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.2740 - val_loss: 18.2016
Epoch 1443/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.6911 - val_loss: 17.9403
Epoch 1444/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.7355 - val_loss: 18.2137
Epoch 1445/10000
4/4 [==============================] - 0s 7ms/step - loss: 7.1503 - val_l

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,12.136589,12.133156,12.12994,12.125576,12.121902,12.117725,12.115011,12.112654,12.112032,12.112887,12.115059,12.115217,12.117489,12.115813,12.114299,12.115675,12.116078,12.117532,12.118626,12.121428,12.124731,12.127186,12.129374,12.13139,12.132431,12.132433,12.131155,12.127023,12.126794,12.12964
Target,13.552,15.826,14.869,14.734,12.754,11.925,13.114,8.92,9.724,9.737,10.136,14.403,12.871,15.749,14.269,15.367,14.595,9.811,11.188,10.381,8.08,11.154,12.508,12.126,14.496,16.723,15.253,16.531,15.206,10.16
Error,1.415411,3.692844,2.73906,2.608424,0.632098,0.192725,0.998989,3.192654,2.388032,2.375887,1.979059,2.287783,0.753511,3.633186,2.154701,3.251325,2.478922,2.306532,0.930626,1.740428,4.044731,0.973186,0.378627,0.005389,2.363569,4.590567,3.121845,4.403977,3.079206,1.96964


In [38]:
display(mae)
display(mape)

2.2227645

0.17578918

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[145.44775]] - Target[149.694]| =  Error: [[4.2462463]]; MAPE:[[0.02836618]]
1/1 [==============================] - 0s 14ms/step
Ano-0: |Prediction[[145.44965]] - Target[148.09900000000002]| =  Error: [[2.649353]]; MAPE:[[0.01788907]]
1/1 [==============================] - 0s 13ms/step
Ano-5: |Prediction[[72.77947]] - Target[88.369]| =  Error: [[15.589531]]; MAPE:[[0.17641401]]


[array([[4.2462463]], dtype=float32),
 array([[2.649353]], dtype=float32),
 array([[15.589531]], dtype=float32)]

7.4950433

0.074223086